In [1]:
import os

# Restrict PyTorch to only see GPU 0
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch

if torch.cuda.is_available():
    print(f"Number of GPUs available: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available, using CPU.")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Number of GPUs available: 1
GPU 0: NVIDIA L40S
Using device: cuda:0


In [3]:
from spatialvla.datasets import RLDSBatchTransform, RLDSDataset
from spatialvla.mobilevlm.model.mobilevlm import load_pretrained_vlm_for_vla, load_vla
from scripts.spatialvla_config import ModelArguments, TrainingArguments
import transformers
from spatialvla.datasets.rlds.utils.data_utils import PaddedCollatorForActionPrediction
from torch.utils.data import DataLoader
from spatialvla.mobilevlm.action_tokenizer import ActionTokenizer
import sys
import torch
import argparse
from PIL import Image
from pathlib import Path
import numpy as np

from spatialvla.mobilevlm.model.mobilevlm import load_vla, load_pretrained_model
from spatialvla.mobilevlm.conversation import conv_templates, SeparatorStyle
from spatialvla.mobilevlm.utils import disable_torch_init, process_images, tokenizer_image_token, KeywordsStoppingCriteria
from spatialvla.mobilevlm.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN
from dataclasses import dataclass
from pathlib import Path
from typing import Any, Dict, Tuple, Type

import numpy as np
import torch
import copy
from PIL import Image

from torch.utils.data import Dataset, IterableDataset

from transformers import PreTrainedTokenizerBase
from transformers import AutoTokenizer, BitsAndBytesConfig

from spatialvla.mobilevlm.utils import disable_torch_init, process_images, tokenizer_image_token, KeywordsStoppingCriteria
# from prismatic.models.backbones.llm.prompting import PromptBuilder
# from prismatic.models.backbones.vision import ImageTransform

from spatialvla.mobilevlm.constants import IGNORE_INDEX, IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
from spatialvla.mobilevlm.conversation import conv_templates, SeparatorStyle
from spatialvla.mobilevlm.model.bimanual import load_twinvla
from spatialvla.datasets.rlds.utils.data_utils import tree_map
# from prismatic.vla.action_tokenizer import ActionTokenizer
from spatialvla.datasets.rlds import make_interleaved_dataset, make_single_dataset
from spatialvla.datasets.rlds.oxe import OXE_NAMED_MIXTURES, get_oxe_dataset_kwargs_and_weights
from spatialvla.datasets.rlds.utils.data_utils import NormalizationType
from transformers import PreTrainedTokenizerBase
import torch.nn as nn

2025-01-24 23:33:14.848639: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-24 23:33:14.878801: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-24 23:33:14.878859: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-24 23:33:14.879837: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-24 23:33:14.885504: I tensorflow/core/platform/cpu_feature_guar

[2025-01-24 23:33:28,804] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [7]:
# model_args = ModelArguments()
# model_args.use_state_input = True

tokenizer, model, image_processor, _ = load_twinvla(
    'checkpoints/handover_kirby_half',
    load_8bit=False, 
    load_4bit=False,
    device='cuda',
    dtype=torch.bfloat16
)

# tokenizer, model, image_processor, _ = load_pretrained_vlm_for_vla(
#     model_args, 
#     load_8bit=False, 
#     load_4bit=False,
#     device='cuda',
#     dtype=torch.bfloat16
# )

at = ActionTokenizer(tokenizer)

batch_transform = RLDSBatchTransform(
    tokenizer,
    image_processor,
    use_state_input=False,
    window_size=1,
    future_action_window_size=7,
    use_hz_input=True
)
vla_dataset = RLDSDataset(
    data_root_dir='/home/shared/rlds_datasets',
    data_mix='handover_kirby',
    batch_transform=batch_transform,
    shuffle_buffer_size=100,
    window_size=1,
    future_action_window_size=7,
    # train=True,
    use_state_input = False
)

collator = PaddedCollatorForActionPrediction(
    tokenizer.model_max_length, 
    tokenizer.pad_token_id, 
    padding_side='right', 
    use_state_input=False,
    use_label=False,
    use_hz_input=True
)

Loading with torch.bfloat16
{'name': 'handover_kirby', 'data_dir': '/home/shared/rlds_datasets', 'image_obs_keys': {'primary': 'rightview_image', 'secondary': 'leftview_image'}, 'absolute_action_mask': [False, False, False, False, False, False, True, False, False, False, False, False, False, True], 'action_normalization_mask': [True, True, True, True, True, True, False, True, True, True, True, True, True, False], 'action_proprio_normalization_type': <NormalizationType.NORMAL: 'normal'>, 'language_key': 'language_instruction', 'standardize_fn': <function lg_delta_ee_transform at 0x1489831a1e10>}


2025-01-24 23:35:01.371786: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2025-01-24 23:35:01.482844: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization



######################################################################################
# Loading the following 1 datasets (incl. sampling weight):                         #
# handover_kirby: ==========================================================1.000000 #
######################################################################################

Threads per Dataset:  [1]
Reads per Dataset:  [1]
Constructing datasets...


2025-01-24 23:35:01.906781: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization


Applying frame transforms on dataset...


In [8]:
dataloader = DataLoader(
    vla_dataset,
    batch_size=1,
    sampler=None,
    collate_fn=collator,
    num_workers=0,  # Important =>> Set to 0 if using RLDS; TFDS rolls its own parallelism!
)

In [256]:
cfg = TrainingArguments()

In [13]:
for d in dataloader:
    batch = d
    if (batch['action'][:, :, 6] != batch['action'][:, :, -1]).sum() > 0:
        break
    else:
        print(batch['action'][:, :, 6], batch['action'][:, :, -1])
device_id = 0

tensor([[0., 0., 0., 0., 0., 0., 0., 0.]], dtype=torch.float16) tensor([[0., 0., 0., 0., 0., 0., 0., 0.]], dtype=torch.float16)
tensor([[4., 4., 4., 4., 4., 4., 4., 4.]], dtype=torch.float16) tensor([[4., 4., 4., 4., 4., 4., 4., 4.]], dtype=torch.float16)
tensor([[4., 4., 4., 4., 4., 4., 4., 4.]], dtype=torch.float16) tensor([[4., 4., 4., 4., 4., 4., 4., 4.]], dtype=torch.float16)
tensor([[4., 4., 0., 0., 0., 0., 0., 0.]], dtype=torch.float16) tensor([[4., 4., 0., 0., 0., 0., 0., 0.]], dtype=torch.float16)
tensor([[0., 0., 0., 0., 0., 0., 0., 0.]], dtype=torch.float16) tensor([[0., 0., 0., 0., 0., 0., 0., 0.]], dtype=torch.float16)
tensor([[4., 4., 4., 4., 4., 4., 4., 4.]], dtype=torch.float16) tensor([[4., 4., 4., 4., 4., 4., 4., 4.]], dtype=torch.float16)
tensor([[4., 4., 4., 4., 4., 4., 4., 4.]], dtype=torch.float16) tensor([[4., 4., 4., 4., 4., 4., 4., 4.]], dtype=torch.float16)
tensor([[4., 4., 4., 4., 4., 4., 4., 4.]], dtype=torch.float16) tensor([[4., 4., 4., 4., 4., 4., 4., 4.]

KeyboardInterrupt: 

In [11]:
batch['action'].shape

torch.Size([1, 8, 14])

In [296]:
with torch.no_grad():
    with torch.autocast('cuda', dtype=torch.bfloat16):
        loss = model.forward(
            input_ids=batch['input_ids'].to(device_id),
            images=batch['pixel_values'].to(device_id),
            attention_mask=batch['attention_mask'].to(device_id),
            actions=batch['action'].to(device_id),
            states=None,
            labels=None,
            hz=batch['hz'].to(device_id)
        )
loss

tensor(2.0465, device='cuda:0')

In [302]:
for d in dataloader:
    batch = d
    break

In [424]:
model.eval()
with torch.no_grad():
    with torch.autocast('cuda', dtype=torch.bfloat16):
        predicted_action = model.predict_action(
            input_ids=batch['input_ids'].to(device_id),
            images=batch['pixel_values'].to(device_id),
            attention_mask=batch['attention_mask'].to(device_id),
            use_cache=True,
            states=None,
            prior_actions=batch['action'].to(device_id),
            num_denoise_steps=10
        )
nn.functional.mse_loss(batch['action'].to(device_id), predicted_action, reduction='mean')

tensor(0.1795, device='cuda:0')

In [425]:
predicted_action[0][1]

tensor([-0.0024,  0.0764,  1.7090, -1.2346,  1.0496,  0.4228, -0.0095],
       device='cuda:0')

In [389]:
1 == True

True

In [427]:
batch['action'][0][1]

tensor([-0.3711, -0.2328,  0.3052, -1.7012,  0.3323,  0.3503,  0.0000],
       dtype=torch.float16)

In [36]:
with torch.autocast('cuda', dtype=torch.bfloat16):
    loss = model.forward(
        input_ids=batch['input_ids'].to(device_id),
        images=batch['pixel_values'].to(device_id),
        attention_mask=batch['attention_mask'].to(device_id),
        actions=batch['action'].to(device_id),
        states=None,
        labels=batch['labels'] if model.config.head_args['head_type'] == 'BR' else None
    )

OutOfMemoryError: CUDA out of memory. Tried to allocate 74.00 MiB (GPU 0; 44.53 GiB total capacity; 42.55 GiB already allocated; 32.00 MiB free; 43.96 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
loss.loss

In [36]:
with torch.autocast('cuda', dtype=torch.bfloat16):
    output_ids = model.generate(
        input_ids[0:1, :-8],
        images=images,
        max_new_tokens=model.config.action_len,
        use_cache=True,
        do_sample=False,
        top_p=None,
    )  

In [18]:
with torch.inference_mode():
    with torch.autocast('cuda', dtype=torch.bfloat16):
        actions_br = model.predict_action(
            input_ids=input_ids[0:1, :-8],
            images=images[0:1],
            num_denoise_steps=5
    )

torch.Size([1, 209, 2048])


AttributeError: 'NoneType' object has no attribute 'cpu'

tensor(0.1394, device='cuda:0')

In [30]:
actions_br[0]

tensor([[-1.1042, -0.3160, -1.5980, -0.0311, -0.8963,  0.4880,  1.0275],
        [-1.2075, -0.3785, -1.6508, -0.0724, -0.9002,  0.4515,  1.0261],
        [-1.3061, -0.3233, -1.4202, -0.0493, -0.0161,  0.4954,  1.0358],
        [-1.3766, -0.2912, -1.3514, -0.1260, -0.0743,  0.6227,  0.9855],
        [-1.3662, -0.3600, -1.0680, -0.0293, -0.1577,  0.1231,  1.0895],
        [-1.1848, -0.1059, -1.0111, -0.0183, -0.2175,  0.2245,  0.0890],
        [-1.0928, -1.0792, -0.9678,  0.0877, -0.2694,  0.1673, -0.0141],
        [-1.0409, -0.0090,  0.0873,  0.0632,  0.7952,  0.0270,  0.0643]],
       device='cuda:0')

In [29]:
actionss[0]

tensor([[-0.9941, -0.3079, -1.4834, -0.0505, -0.8149,  0.3569,  1.0000],
        [-1.1533, -0.3079, -1.4473, -0.0505, -0.5762,  0.3569,  1.0000],
        [-1.2539, -0.3079, -1.3994, -0.0505, -0.4023,  0.3569,  1.0000],
        [-1.3936, -0.3079, -1.2979, -0.0505, -0.2937,  0.3569,  1.0000],
        [-1.4336, -0.3201, -1.2021, -0.0505, -0.1418,  0.2805,  1.0000],
        [-1.3037, -0.3872, -1.0469, -0.0505, -0.1418,  0.1787,  0.0000],
        [-0.9941, -0.4360, -0.7231, -0.0505, -0.0333,  0.0261,  0.0000],
        [-0.5244, -0.3628, -0.2622, -0.0505,  0.6182, -0.0757,  0.0000]],
       device='cuda:0', dtype=torch.float16)

In [14]:
prior_action = torch.tensor(at.detokenize(input_ids[0:3, -8:].cpu().numpy()))

In [15]:
with torch.inference_mode():
    with torch.autocast('cuda', dtype=torch.bfloat16):
        actions = model.predict_action(
            input_ids=input_ids[0:3],
            images=images[0:3],
            prior_actions=prior_action,
        )
    
# condition = model.condition_projector(action_hidden)
# predicted_action = model.si.sample(
#     x_prior=prior_action.cuda().to(dtype=torch.bfloat16),
#     cond=condition.float().flatten(1),
#     diffuse_step=5
# )